## Example - 2-fluid problem with no outflow boundary conditions - MMS

This notebook generates the Method of Manufactured Solutions test from the ReMKiT1D paper. See section 5.1.2. in that paper for more details.

This is the v2.0.0 rewrite of the original notebook.

**NOTE**: Due to changes to some operator conventions in v2.0.0 the exact numerical values of the errors from the paper are not exactly replicated, but the MMS convergence is reproduced, as expected.

In [ ]:
import RMK_support as rmk 
import RMK_support.common_models as cm
import RMK_support.dashboard_support as ds

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt
import panel as pn
import pickle

### Some useful constants

In [ ]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27 #atomic mass unit
ionMass = 2.014*amu # deuterium mass
epsilon0 = 8.854188e-12 #vacuum permittivity 
heavySpeciesMass = 2.014 #in amus

### Context initialisation 

In [ ]:
rk = rmk.RMKContext()

rk.mpiContext = rmk.MPIContext(8)
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_MMS_test/")

### Grid initialisation

In [ ]:
L=10
Nx = 1024
xGridWidths = L/Nx*np.ones(Nx)
rk.grid = rmk.Grid(xGridWidths,interpretXGridAsWidths=True,isLengthInMeters=True)

### Species data

In [ ]:
electronSpecies = rmk.Species("e",0,atomicA=elMass/amu,charge=-1.0) 

ionSpecies = rmk.Species("D+",-1,atomicA=2.014,charge=1.0)

rk.species.add(electronSpecies,ionSpecies)

### Calculate variable initial values based on MMS test

In [ ]:
# Manufactured solution terms

lengthNorm = rk.norms["length"]
dualGrid = rk.grid.xGridDual
dn = 0.1
n_MMS = np.ones(Nx) + dn * (rk.grid.xGrid-L)/L
ndual_MMS = np.ones(Nx) + dn * (dualGrid-L)/L

T0 = 0.5
u0 = 0.01

u_MMS = -u0 * rk.grid.xGrid*(rk.grid.xGrid-L)/L**2
udual_MMS = -u0 *  dualGrid*((dualGrid-L)/L)/L
dudx = -u0*(2*rk.grid.xGrid-L)/L**2*lengthNorm
dudualdx = -u0*(2*dualGrid-L)/L**2*lengthNorm

gamma_MMS = n_MMS * u_MMS 
gammadual_MMS = ndual_MMS*udual_MMS

dndx = dn/L*lengthNorm

dGamma = dndx*u_MMS+dudx*n_MMS
dGammadual = dndx*udual_MMS + dudualdx*ndual_MMS
duGammadual = dGammadual*udual_MMS + dudualdx*gammadual_MMS

fixBoundaryCells = False
# Numerically modify duGammadual to account for fact ReMKiT1D uses extended boundary cells on staggered grid
if fixBoundaryCells:
    duGammadual[0] = (udual_MMS[0]*gammadual_MMS[0]+gammadual_MMS[1]*udual_MMS[1])/2 *lengthNorm/ (xGridWidths[0]+xGridWidths[1]/2) 
    duGammadual[-2] = -(udual_MMS[-2]*gammadual_MMS[-2]+udual_MMS[-3]*gammadual_MMS[-3])/2 *lengthNorm/ (xGridWidths[-1]+xGridWidths[-2]/2) 

T_MMS = T0*np.ones(Nx)

Edual_MMS = - (0.5 * T_MMS * dndx + duGammadual)/ndual_MMS # Using assumed normalization

### Set variables

In [ ]:
ne,ne_dual = rmk.varAndDual("ne",rk.grid,data=n_MMS,units='$10^{19} m^{-3}$')

ni,ni_dual = rmk.varAndDual("ni",rk.grid,data=n_MMS,units='$10^{19} m^{-3}$')

Ge_dual,Ge = rmk.varAndDual("Ge",rk.grid,data=gammadual_MMS,primaryOnDualGrid=True)

Gi_dual,Gi = rmk.varAndDual("Gi",rk.grid,data=gammadual_MMS,primaryOnDualGrid=True)

Te,Te_dual = rmk.varAndDual("Te",rk.grid,data=T_MMS,units='$10eV$')

E_dual,E = rmk.varAndDual("E",rk.grid,data=Edual_MMS,primaryOnDualGrid=True)

ue_dual,ue = rmk.varAndDual("ue",rk.grid,primaryOnDualGrid=True,derivation=rk.textbook["flowSpeedFromFlux"],derivationArgs=["Ge_dual","ne_dual"])

ui_dual,ui = rmk.varAndDual("ui",rk.grid,primaryOnDualGrid=True,derivation=rk.textbook["flowSpeedFromFlux"],derivationArgs=["Gi_dual","ni_dual"])

rk.variables.add(ne,ne_dual,ni,ni_dual,Ge_dual,Ge,Gi_dual,Te,Te_dual,E_dual,E,ue_dual,ue,ui_dual,ui)

### Models

In [ ]:
#Advection and pressure gradient

rk.models.add(cm.advection(ne,Ge_dual).rename("continuity-ne"),
              cm.advection(ni,Gi_dual).rename("continuity-ni"),
              cm.pressureGrad(Ge_dual,Te*ne,normConst=0.5).rename("pressureGrad-Ge"),
              cm.pressureGrad(Gi_dual,Te*ni,normConst=elMass/(2*ionMass)).rename("pressureGrad-Gi"),
              cm.advection(Ge_dual,ue_dual*Ge_dual).rename("advection-Ge"),
              cm.advection(Gi_dual,ui_dual*Gi_dual).rename("advection-Gi"))

In [ ]:
#Ampere-Maxwell and Lorentz force

rk.models.add(cm.ampereMaxwell(E_dual,
                               speciesFluxes=[Ge_dual,Gi_dual],
                               species=[electronSpecies,ionSpecies],
                               norms=rk.norms).rename("ampereMaxwell"),
              cm.lorentzForces(E_dual,
                               speciesFluxes=[Ge_dual,Gi_dual],
                               speciesDensities=[ne_dual,ni_dual],
                               species=[electronSpecies,ionSpecies],
                               norms=rk.norms).rename("lorentzForce"))

### MMS Source model

In [ ]:
nSource = rk.grid.profile(dGamma,latexName="S_{n,MMS}")

gammaSourceion = rk.grid.profile(duGammadual +  (0.5*T_MMS * dndx - ndual_MMS*Edual_MMS)*elMass/ionMass,latexName="S_{\\Gamma,MMS}")

mmsModel = rmk.Model("mmsModel")

# Continuity equation MMS sources
mmsModel.ddt[ne] += cm.simpleSourceTerm(ne,nSource).rename("sourcene")
mmsModel.ddt[ni] += cm.simpleSourceTerm(ni,nSource).rename("sourceni")


# Momentum equation MMS source

mmsModel.ddt[Gi_dual] += ni_dual**(-1) * (gammaSourceion * rmk.DiagonalStencil()(ni)).rename("sourceGi")

rk.models.add(mmsModel)

### Integration scheme setup

In [ ]:
integrator = rmk.BDEIntegrator("BDE",nonlinTol=1e-14,absTol=10.0,convergenceVars=[ne,ni,Ge_dual,Gi_dual])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models) 
rk.integrationScheme = rmk.IntegrationScheme(dt=rmk.Timestep(10.0*Te**1.5/ne),steps=integrationStep) 
rk.integrationScheme.setOutputPoints(list(np.linspace(1000,20000,20)))

### Add term diagnosis variables

In [ ]:
rk.addTermDiagnostics(ne,Ge_dual,Gi_dual,E_dual)

### Write config file

In [ ]:
rk.writeConfigFile()

### Generate PDF summary

In [ ]:
rk.generatePDF("MMS test")

## Data analysis

In [ ]:
loadedData = rk.loadSimulation().dataset
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

#### Explore data using basic dashboard

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show()


In [ ]:
varsToCheck = ['ne','ue_dual','E_dual']
mmsVars = [n_MMS,udual_MMS,Edual_MMS]

errors = [np.abs(loadedData[varsToCheck[i]]-mmsVars[i])/abs(mmsVars[i]) for i in range(len(mmsVars))]
maxErrors = [error.reduce(np.nanmax,dim) for error,dim in zip(errors,["x","x_dual","x_dual"])]

In [ ]:
curves = hv.Overlay([hv.Curve(maxErrors[i],label=varsToCheck[i]) for i in range(len(mmsVars))])

In [ ]:
curves.opts(ylim=(1e-10,1),logy=True)

In [ ]:
maxErrorDict = {}
maxErrorDictFix = {}

#### Add maximum errors based on which run was performed

To reproduce the results, run ReMKiT1D with the appropriate fixBoundaryCells flag and the number of spatial cells Nx

In [ ]:
if fixBoundaryCells:
    maxErrorDictFix[Nx]=maxErrors
else:
    maxErrorDict[Nx]=maxErrors

In [ ]:
pickle.dump((maxErrorDict,maxErrorDictFix),open("mmsTest.pkl","wb"))

In [ ]:
maxErrorDict,maxErrorDictFix= pickle.load(open("mmsTest.pkl","rb"))

In [ ]:
plotNoFix = hv.Overlay([hv.Curve((list(maxErrorDict.keys()),[maxErrorDict[key][i][-1] for key in maxErrorDict.keys()])) for i in range(len(mmsVars))])
plotNoFix.opts(logx=True,logy=True,xlabel='$N_x$',ylabel='max($\delta$)',ylim=(1e-9,1))

In [ ]:
plotFix = hv.Overlay([hv.Curve((list(maxErrorDictFix.keys()),[maxErrorDictFix[key][i][-1] for key in maxErrorDictFix.keys()]),label=varsToCheck[i]) for i in range(len(mmsVars))])
plotFix.opts(logx=True,logy=True,xlabel='$N_x$',ylabel='max($\delta$)',ylim=(1e-9,1))

In [ ]:
hv.save(plotNoFix, 'mmsTestNoFix.pdf', dpi=144)
hv.save(plotFix, 'mmsTestFix.pdf', dpi=144)


### Number of acoustic transition times the simulation is run for

In [ ]:
20000*lengthNorm*np.sqrt(T0*elMass/ionMass)/L